In [1]:
# import libraries
import sys, os, signal, time, threading, json, webbrowser

import jupyros
import rospy, message_filters, actionlib
from std_msgs.msg import String, Header
from actionlib_msgs.msg import GoalStatus

! source /home/qcr/cgras_moveit_ws/devel/setup.bash
sys.path.append('/home/qcr/cgras_moveit_ws/devel/lib/python3/dist-packages')

# project modules
import cgras_robot.msg
from cgras_robot.msg import CalibrateAction, MoveAction, ResetAction, CalibrateGoal, MoveGoal, ResetGoal

rospy.init_node('action_node')

In [2]:
jupyros.action_client('/cgras/robot/move', MoveAction, MoveGoal, callbacks={})

[INFO] [1703991446.045577]: [/CGRAS/ROBOT/MOVE] Waiting for action server.
[INFO] [1703991446.059186]: [/CGRAS/ROBOT/MOVE] Connection to server successful.


[INFO] [1703991453.252157]: [/CGRAS/ROBOT/MOVE] Result is error: ''


[ERROR] [1704075608.238758]: Got a transition callback on a goal handle that we're not tracking


[INFO] [1704075608.310078]: [/CGRAS/ROBOT/MOVE] Result is error: ''


In [3]:
jupyros.action_client('/cgras/robot/reset', ResetAction, ResetGoal, callbacks={})

[INFO] [1703991449.542732]: [/CGRAS/ROBOT/RESET] Waiting for action server.
[INFO] [1703991449.564212]: [/CGRAS/ROBOT/RESET] Connection to server successful.


[INFO] [1703991456.464439]: [/CGRAS/ROBOT/RESET] Result is error: ''


In [ ]:
def cb_calibrate_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print(True, f'success: the location apriltag found {result.location}')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')   

def cb_move_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print.ACTION_RESULT.set_result(True, f'success: moved to the grid coordinate')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')   

def cb_reset_done(status, result):
    if status == GoalStatus.SUCCEEDED:
        print.ACTION_RESULT.set_result(True, f'success: reset to the desired pose')
    elif status == GoalStatus.ABORTED:
        print.ACTION_RESULT.set_result(False, f'error: the error is {result.error}')              

# -- callback from the GUI console
def _client_callback(event, *args):
    if event == 'calibrate':
        action_client = actionlib.SimpleActionClient('/cgras/robot/calibrate', 
                                                            CalibrateAction)
        action_client.wait_for_server()
        goal = CalibrateGoal()
        action_client.send_goal(goal, done_cb=cb_calibrate_done)
            
    elif event == 'move':
        action_client = actionlib.SimpleActionClient('/cgras/robot/move', 
                                                            MoveAction)
        tile_x = int(args[0])
        tile_y = int(args[1])
        grid_x = int(args[2])
        grid_y = int(args[3])
        action_client.wait_for_server()
        goal = MoveGoal(tile_x=tile_x, tile_y=tile_y, grid_x=grid_x, grid_y=grid_y)
        action_client.send_goal(goal, done_cb=cb_move_done)

    elif event == 'reset':
        action_client = actionlib.SimpleActionClient('/cgras/robot/reset', 
                                                            ResetAction)
        pose = args[0]
        action_client.wait_for_server()
        goal = ResetGoal(pose=pose)
        action_client.send_goal(goal, done_cb=cb_reset_done)      